### Cleaning and creating annual table in the database


1. Required Libraries

In [1]:
import pandas as pd
import os

2. Path to the directory of the csv files and their concatenation

In [3]:
# Accessing the file directory
route = "path\\to\\the\\file\\folder"

# Creating a list with all csv files
files_csv = [os.path.join(route, file) for file in os.listdir(route) if file.endswith('.csv')]

# Reading and concatenating files
dfs = [pd.read_csv(file) for file in files_csv]
df_total = pd.concat(dfs, ignore_index=True)

print("Files read and concatenated successfully.")

Files read and concatenated successfully.


3. Data verification

In [4]:
df_total.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,903C30C2D810A53B,electric_bike,2023-08-19 15:41:53,2023-08-19 15:53:36,LaSalle St & Illinois St,13430,Clark St & Elm St,TA1307000039,41.890721,-87.631477,41.902973,-87.63128,member
1,F2FB18A98E110A2B,electric_bike,2023-08-18 15:30:18,2023-08-18 15:45:25,Clark St & Randolph St,TA1305000030,NaN,NaN,41.884511,-87.631550,41.930000,-87.64000,member
2,D0DEC7C94E4663DA,electric_bike,2023-08-30 16:15:08,2023-08-30 16:27:37,Clark St & Randolph St,TA1305000030,NaN,NaN,41.884981,-87.630793,41.910000,-87.63000,member
3,E0DDDC5F84747ED9,electric_bike,2023-08-30 16:24:07,2023-08-30 16:33:34,Wells St & Elm St,KA1504000135,NaN,NaN,41.903105,-87.634667,41.900000,-87.62000,member
4,7797A4874BA260CA,electric_bike,2023-08-22 15:59:44,2023-08-22 16:20:38,Clark St & Randolph St,TA1305000030,NaN,NaN,41.885548,-87.632019,41.890000,-87.68000,member


There are text strings, numbers, dates and times.

4. Review of data types

In [5]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5715693 entries, 0 to 5715692
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 566.9+ MB


There is data in an incorrect format. For example, columns with dates and times.

5. Removed possible milliseconds and converted to 'datetime' data type for columns that require it

In [6]:
# Remove milliseconds and convert to date and time
df_total['started_at'] = df_total['started_at'].str.split('.', n=1).str[0]
df_total['started_at'] = pd.to_datetime(df_total['started_at'], format="%Y-%m-%d %H:%M:%S")
df_total['ended_at'] = df_total['ended_at'].str.split('.', n=1).str[0]
df_total['ended_at'] = pd.to_datetime(df_total['ended_at'], format="%Y-%m-%d %H:%M:%S")

print("Modifications completed successfully.")

Modifications completed successfully.


6. Remove any extra white space

In [7]:
# Removing whitespace from 'object' type columns and tracking how many were removed
for col in df_total.columns:
    if df_total[col].dtype == 'object':
        # Count how many values ​​have spaces at the beginning or end
        spaces_before = df_total[col].str.match(r'^\s+|\s+$').sum()

        if spaces_before > 0:
            # Remove blank spaces
            df_total[col] = df_total[col].str.strip()
            print(f"Remove {spaces_before} spaces in column: {col}")

Remove 2 spaces in column: start_station_id
Remove 3 spaces in column: end_station_id


7. Checking for empty or NaN values ​​in 'object' type columns

In [8]:
# Filter columns of object type
columns = df_total.select_dtypes(include='object')
object = columns.isna().sum()
print(object)

ride_id                    0
rideable_type              0
start_station_name    947025
start_station_id      947025
end_station_name      989476
end_station_id        989476
member_casual              0
dtype: int64


There are NaN or empty values ​​in several columns.

8. Transformation of NaN values ​​to NULL

In [9]:
# Reemplazando NaN por None 
df_total['start_station_name'] = df_total['start_station_name'].where(pd.notna(df_total['start_station_name']), None)
df_total['start_station_id'] = df_total['start_station_id'].where(pd.notna(df_total['start_station_id']), None)
df_total['end_station_name'] = df_total['end_station_name'].where(pd.notna(df_total['end_station_name']), None)
df_total['end_station_id'] = df_total['end_station_id'].where(pd.notna(df_total['end_station_id']), None)
print("Data successfully transformed.")

Data successfully transformed.


Clarification: The changes will not be reflected in the dataframe but will be reflected internally. When loading them into the database they will be reflected as 'NULL'.

9. Checking for empty or NaN values ​​in 'float' type columns

In [9]:
# Count the NaNs in each column
nan_counts = df_total[['start_lat', 'start_lng', 'end_lat', 'end_lng']].isna().sum()
print(nan_counts)

start_lat       0
start_lng       0
end_lat      7756
end_lng      7756
dtype: int64


There are NaN values ​​in the 'end_lat' and 'end_lng' columns.

10. Transforming NaN values ​​to 0

In [10]:
# Replacing NaN with 0
df_total['end_lat'] = df_total['end_lat'].fillna(0)
df_total['end_lng'] = df_total['end_lng'].fillna(0)

In this case, the changes will be reflected, since we changed the values ​​to 0.

11. Connecting to the database with 'sqlalchemy'

In [17]:
from sqlalchemy import create_engine

# Creating database connection
conexion_str = f"mysql+pymysql://{'user'}:{'password'}@{'name_host'}:{'host in number'}/{'name_database'}"
engine = create_engine(conexion_str)

12. Creating a table in the database

We create a table in the database and load the dataframe in blocks of 10,000 rows.

In [18]:
# Declaring the table name
table_name = 'last_year_report'  

# Loading the dataframe in blocks of 10000
block_size = 10000
for i, start_row in enumerate(range(0, len(df_total), block_size)):
    end_row = min(start_row + block_size, len(df_total))
    df_block = df_total.iloc[start_row:end_row]
    df_block.to_sql(table_name, con=engine, if_exists='append', index=False)
    print(f'Block {i+1} loaded successfuly.')

print('All blocks have been loaded.')

Block 1 loaded successfuly.
Block 2 loaded successfuly.
Block 3 loaded successfuly.
Block 4 loaded successfuly.
Block 5 loaded successfuly.
Block 6 loaded successfuly.
Block 7 loaded successfuly.
Block 8 loaded successfuly.
Block 9 loaded successfuly.
Block 10 loaded successfuly.
Block 11 loaded successfuly.
Block 12 loaded successfuly.
Block 13 loaded successfuly.
Block 14 loaded successfuly.
Block 15 loaded successfuly.
Block 16 loaded successfuly.
Block 17 loaded successfuly.
Block 18 loaded successfuly.
Block 19 loaded successfuly.
Block 20 loaded successfuly.
Block 21 loaded successfuly.
Block 22 loaded successfuly.
Block 23 loaded successfuly.
Block 24 loaded successfuly.
Block 25 loaded successfuly.
Block 26 loaded successfuly.
Block 27 loaded successfuly.
Block 28 loaded successfuly.
Block 29 loaded successfuly.
Block 30 loaded successfuly.
Block 31 loaded successfuly.
Block 32 loaded successfuly.
Block 33 loaded successfuly.
Block 34 loaded successfuly.
Block 35 loaded success

At this point, we have completed the data cleaning phase and can now work on the database.